<center> <img src="res/ds3000.png"> </center>

<center> <h1> Week 5 - Day 1 </h1> </center>

<center> <h2> Part 1: Web Crawling </h2></center>

## Outline
1. <a href='#1'>Review of Web Scraping using BeautifulSoup</a>
2. <a href='#2'>Web Crawling</a>
3. <a href='#3'>More on Web Scraping</a>


<a id="1"></a>

## 1. Review of Web Scraping using BeautifulSoup

In [1]:
import urllib.request as urllib
from bs4 import BeautifulSoup
import pandas as pd

In [2]:

html = urllib.urlopen("https://www.khoury.northeastern.edu/role/tenured-and-tenure-track-faculty/")
soup = BeautifulSoup(html.read())

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1056)>

In [ ]:


#retrieves all <div class = "grid-item"> tags
faculty_divs = soup.find_all("div", class_="grid-item")


df = pd.DataFrame(columns = ["Name", "Title", "Link", "Picture"])

for prof in faculty_divs:   
    fname = prof.find("h3").get_text().strip()
    ftitle= prof.find("p","position").get_text().strip()
    fpage = prof.find("a").get("href").strip()
    fpic = prof.find("img").get("src").strip()
    
#appends the fields to their respective columns
#note the curly braces 
    df = df.append({"Name":fname, "Title":ftitle, "Link":fpage, "Picture": fpic}, ignore_index=True)
    df.to_csv("khoury_faculty.csv")

<a id="2"></a>

## 2. Web Crawling
* Suppose we want to scrape Dean Brodley's email address from the Khoury website.
    * Let's take a look at Dean Brodley's homepage:
    * https://www.khoury.northeastern.edu/people/carla-brodley/

In [ ]:
import urllib.request as urllib
from bs4 import BeautifulSoup

deanURL = "https://www.khoury.northeastern.edu/people/carla-brodley/"
    
dean_page = urllib.urlopen(deanURL)
page_soup = BeautifulSoup(dean_page.read())

In [ ]:
page_soup

<img src = "res/email.png" />

* Let's scrape Dean Brodley's email address from this page:

In [ ]:
import urllib.request as urllib
from bs4 import BeautifulSoup

deanURL = "https://www.khoury.northeastern.edu/people/carla-brodley/"

#let's open the page
dean_page = urllib.urlopen(deanURL)

#creates a BeautifulSoup object containing the content of the page
page_soup = BeautifulSoup(dean_page.read())

#finds the p tage with class = "contact-email"
email_container = page_soup.find("p", class_="contact-email")

#finds the a tag the within the contact-email p tag and extracts the text(email address)
email = email_container.find("a").get_text()

In [ ]:
email

### Let's do this for everyone on the page!

### 2.1. Web Crawling
* Scrapers traversing multiple pages and even multiple sites
* Web crawlers retrieve page contents for a URL, examine that page for another URL, and retrieve that page or some portions of it

In [ ]:
import urllib.request as urllib
from bs4 import BeautifulSoup
import time
import random

#opens the faculty page
html = urllib.urlopen("https://www.khoury.northeastern.edu/role/tenured-and-tenure-track-faculty/")
#turns the pag into a BeautifulSoup object
soup = BeautifulSoup(html.read())

#get all the content under <div class = "grid-item">
faculty_divs = soup.find_all("div", class_="grid-item")

#defines an empty list that will contain the email addressed crawled from faculty webpages
emails = []

for prof in faculty_divs:
    
    #gets the URL to their webpage
    fpage = prof.find("a").get("href").strip()
    
    #opens the URL for each prof
    fac_page = urllib.urlopen(fpage)    
    #turns the URL for each prof into a BeautifulSoup object
    page_soup = BeautifulSoup(fac_page.read())
    
    #closes the urllib connection so the website won't get mad at us
    fac_page.close()
    
    #on the new page, finds the email container, <p class = "contact_email">
    email_container = page_soup.find("p", class_="contact-email")
    #gets the text for the <a> tage, the email address
    email = email_container.find("a").get_text()
    
    #appends the email to a list and displays it
    emails.append(email)
    print(email)
    
    #waits for a random number of seconds(2-5) before moving on to the next prof in the iterable
    #done to avoid overwhelming the website and getting blocked as a bot
    time.sleep(random.randint(2,6))

print("\n\n\nDone scraping the addresses")

In [ ]:
#displays the list of email addresses
emails

#### Let's add these email addresses to our dataframe, df

In [ ]:
df.head()

In [ ]:
len(emails)

In [ ]:
#we can add a new column
df["Email"] = emails

In [ ]:
df.head()

<a id="3"></a>

## 3. More on Web Scraping
* **BeautifulSoup** Documentation: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
* **Selenium:** https://selenium-python.readthedocs.io/index.html
    * Web automation and scraping; dynamic GET and POST requests; can interact with dynamic web pages, forms, etc.
* **Scrapy:** https://scrapy.org/
    * Optimized web crawling tasks